In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
from IPython.display import clear_output
import os

import scipy
import matplotlib

from robot import *
from utils import *
import pybullet as p
import pybullet_data
import networkx as nx
from functools import partial

%load_ext autoreload
%autoreload 2

## Setup Pybullet

In [2]:
# name_in =  'urdf/support.obj'
# name_out = 'urdf/support_concave.obj'
# name_in =  'urdf/support_cylindrical_col.obj'
# name_out = 'urdf/support_cylindrical_colconc.obj'
# name_log = "log.txt"
# p.vhacd(name_in, name_out, name_log, alpha=0.04,resolution=1000000 )

Now 'urdf/support_cylindrical.urdf' shows the structure real visualization but with the collision shape constructed from the command p.vhacd(.)

In [3]:
physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)


In [4]:
p.resetSimulation()

### Setup Robot

In [5]:
robot_urdf = 'urdf/panda/panda_arm_rivet.urdf';
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)

plane_id = p.loadURDF('plane.urdf')
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id, pb_joint_indices)


set_q_std = partial(set_q,robot_id, pb_joint_indices)

vis_traj_std = partial(vis_traj, vis_func= set_q_std)

### Visualize the big frame

In [6]:
is_simple = False
obs_ids = []
rgbacolor = (0,0,1,0)

board_id = p.loadURDF('urdf/support_cylindrical.urdf', globalScaling=0.0008, useFixedBase=1)
quat = w2quat(np.array([0,0,-np.pi/2]))
p.resetBasePositionAndOrientation(board_id, (0.6,-2.2,-.3), quat)

offset = 0.

In [7]:
x_board = 0.37
p.resetBasePositionAndOrientation(board_id, (x_board,-1.7,-.41), quat)

### Setup targets in the middle

In [8]:
ys = np.concatenate([np.arange(-0.34, -0.15, 0.05), np.arange(0.2, 0.36, 0.05)],-1)
zs = np.arange(0.35, 0.54, 0.05) + 0.2
x = x_board + 0.3
targets_mid = []
target_ids = []
offset = -0.05
for y in ys:
    for z in zs:
        target = np.array([x, y,z])
        targets_mid += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(1,0,0,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_mid[-1], (0,0,0,1))
        targets_mid[-1][0] -= 0.03

targets_mid = np.array(targets_mid)

### Targets on the bottom

In [9]:
R = w2mat(quat2w(euler2quat([0,10,0.])))

In [10]:
zs = np.arange(0.25,0.31, 0.05) + 0.2
#ys = np.array([-0.45, -0.25,  0.15,  0.35])
#zs = np.arange(0.6, 0.81, 0.1)
x = x_board + 0.25
targets_down = []
target_ids = []
offset = 0.03
for y in ys:
    offset_x = 0.
    for z in zs:
        target = np.array([x+offset_x, y,z])
        targets_down += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(0,0,1,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_down[-1], (0,0,0,1))
        targets_down[-1] += (R@np.array([1,0,0]))*offset
        offset_x += 0.03

targets_down = np.array(targets_down)

In [11]:
for target_id in target_ids:
    p.resetBasePositionAndOrientation(target_id,posObj=p.getBasePositionAndOrientation(target_id)[0 ], ornObj=euler2quat([0,10,0.]).tolist())

### Targets on the top

In [12]:
R = w2mat(quat2w(euler2quat([0,-10,0.])))

In [13]:
zs = np.arange(0.55,0.61, 0.05) + 0.2

x = x_board + 0.27
targets_up = []
target_ids = []
offset = 0.03
for y in ys:
    offset_x = 0.
    for z in zs:
        target = np.array([x+offset_x, y,z])
        targets_up += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(0,1,0,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_up[-1], (0,0,0,1))
        targets_up[-1] += (R@np.array([1,0,0]))*offset
        offset_x -= 0.03

targets_up = np.array(targets_up)

In [14]:
for target_id in target_ids:
    p.resetBasePositionAndOrientation(target_id,posObj=p.getBasePositionAndOrientation(target_id)[0], ornObj=euler2quat([0,-10,0.]).tolist())

### RRT

In [18]:
rob_sampler = sampler(joint_limits)
# rob_col_checker = col_checker(robot_id, pb_joint_indices, [ plane_id] + obs_ids)
rob_col_checker = col_checker(robot_id, pb_joint_indices, [ plane_id] + [board_id])

rob_interpolator = interpolator()

In [19]:
rrt = BiRRT(7, rob_sampler, rob_col_checker, rob_interpolator)

In [20]:
start_state = rrt.sample(True).flatten()
end_state = rrt.sample(True).flatten()
traj,_,_ = rrt.plan(start_state,end_state)
shortpath = rrt.shortcut_path(traj)
shorttraj = rrt.interpolate_traj(shortpath, traj)

Goal is reached!
Solution found!


In [21]:
for q in shorttraj[:2]:
    set_q_std(q)
    time.sleep(0.05)

#### Setup IK solver

In [22]:
class CostProjector():
    def __init__(self, cost):
        self.cost = cost
    
    def project(self, q, ftol = 1e-12, gtol = 1e-12, disp = 1, maxiter = 1000 ):
        #update the variables
        self.cost.reset_iter()

        status = False
        res = minimize(self.cost.calc,q,method='l-bfgs-b', jac=self.cost.calcDiff,  options={'ftol':ftol, 'gtol':gtol, 'disp':disp, 'maxiter':maxiter})      
        res = {'stat':status, 'q':  self.cost.qs[-1], 'qs': self.cost.qs, 'nfev': self.cost.nfev, 'feval': self.cost.feval}
        return res

#### Load model in pinocchio

In [23]:
robot_urdf = 'urdf/panda/panda_arm_rivet.urdf';
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()

pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_grasptarget_hand')
pb_ee_frame_id = 7

In [24]:
# Teguh version
# robot_urdf = 'urdf/panda/panda_arm_franka.urdf';
# rmodel = pin.buildModelFromUrdf(robot_urdf)
# rdata = rmodel.createData()

# pin_frame_names = [f.name for f in rmodel.frames]
# ee_frame_id = rmodel.getFrameId('panda_grasptarget_hand')
# pb_ee_frame_id = 7

#### Set IK Solver

In [25]:
x = np.array([0.4,0.2,0.4])
w = np.array([0.,np.pi/2,0.])
pose = np.concatenate((x,w))

#define joint limit cost
bound_cost = CostBound(joint_limits)
#define pose cost
pos_cost = CostFrame(rmodel, rdata, pose, ee_frame_id, np.array([1,1,1,1,1,1]))

#Define IK solver
cost_sum_ik = CostSum()
cost_sum_ik.addCost(pos_cost, 50., 'pose_cost', 1e-12)
cost_sum_ik.addCost(bound_cost, 100., 'joint_limit', 1e-12)
ik_solver = CostProjector(cost_sum_ik)

## Load Data

In [85]:
# Loading the rivet positions
targets_x = np.load("targets_x.npy", allow_pickle=True)
targets_orn = np.load("targets_orn.npy", allow_pickle=True)
rotation_axes = np.load('rotation_axes.npy', allow_pickle=True)
labels = targets_x.item().keys()
hole_nb = 0
for label in labels:
    hole_nb += targets_x.item()[label].shape[0]
print(hole_nb)
targets = np.concatenate([targets_x.item()[name] for name in labels],0)
# Loading the joint configurations
data = np.load('riveting_results.npy', allow_pickle=True)[()]
qs_raw = data['qs']
disp = np.linalg.norm(data['disp'], axis=2)
disp = data['disp'][:,:,0]

64


#### Filter the data

In [86]:
rob_col_checker.check_collision(qs_raw[0,0])

True

In [87]:
qs = []
disps = []
count_col = 0
for i,qset in enumerate(qs_raw):
    qlist = []
    displist = []
    for j,q in enumerate(qset):
        if rob_col_checker.check_collision(q) is False:
            qlist += [q]
            displist += [disp[i][j]*1000]
        else:
            count_col += 1 
#             print('Collision',count_col)
    qlist = np.array(qlist)
    displist = np.array(displist)
    if len(qlist)== 0: 
        print(i)
        
    qs += [qlist]
    disps += [displist]
clear_output()

In [88]:
rrt.check_collision(qs[0][0])

False

In [90]:
i = 0
for q in qs:
    if len(q) == 0: print(i)
    i += 1

49
61


In [34]:
j = 0

In [83]:
rob_col_checker.check_collision(qs_raw[61,j])
j+=1

#### Visualize the data

### Construct Graph & Solve TSP

In [ ]:
from rtsp import *

In [ ]:
graph = from_coordinate_list(targets, euclidean_metric)

path = two_opt(graph)

### Construct the c-space graph

In [ ]:
qs.shape

In [ ]:
max_config = 50

q_init = np.array([ 0.73170025, -1.10558688,  0.22936525, -2.28523573, -1.07928728,
        2.17303381, -1.24231352])[None,:]

q_init = qs[0][7][None,:]
set_q_std(q_init[0])

In [ ]:
class time_struct():
    def __init__(self):
        self.select_config = 0
        self.plan = 0
        self.postplan = 0

In [ ]:
def plan_motion(q_init, path, qs, disps, max_config, w_disp = 20, max_try = 20):
    comp_time = time_struct()
    tic = time.time()
    print('Obtaining configurations in the required order')
    qs_new = [q_init]
    disps_new = [np.array([0])]
    for idx in path:
        qs_new += [qs[idx][:max_config]]
        disps_new += [disps[idx][:max_config]]
    qs_new += [q_init]
    disps_new += [np.array([0])]
    
    print("Construct cgraph")
    cgraph = construct_cgraph(qs_new, disps_new, w_disp = w_disp)
    
    print('Plan configuration path')
    cpath = nx.dijkstra_path(cgraph, 0, len(cgraph.nodes()) - 1)
    traj, total_disp = extract_traj(cgraph, cpath)
    clear_output()
    toc = time.time()
    
    comp_time.select_config = toc-tic
    comp_time.plan = 0
    comp_time.postplan = 0
    
    print('Plan motion')
    trajs = []
    total_dist = 0
    for i in range(0,len(cpath)-1):
        q = cgraph.nodes[cpath[i]]['value']
        qn = cgraph.nodes[cpath[i+1]]['value']
        tic = time.time()
        traj,_,_ = rrt.plan(q,qn)
        toc = time.time()
        comp_time.plan += toc-tic
        shortpath = rrt.shortcut_path(traj, max_try = max_try)
        shorttraj = np.array(traj)[shortpath]
        shortpath2 = rrt.shortcut_path_old(shorttraj,step_length=0.05)
        shorttraj2 = rrt.interpolate_traj(shortpath2, shorttraj)
        toc2 = time.time()
        comp_time.postplan += toc2-toc
        trajs += [shorttraj2]
        total_dist += calc_dist(shorttraj2)
        clear_output()

    clear_output()
    print('Planning time:{}, {}, {}'.format(comp_time.select_config, comp_time.plan, comp_time.postplan))
    return trajs, total_dist, total_disp, comp_time

In [ ]:
def plan_motion_max(q_init, path, qs, disps, max_config, w_disp = 0.3, max_try = 20):
    comp_time = time_struct()
    tic = time.time()
    print('Obtaining configurations that minimizes d') 
    qs_new = [q_init[0]]
    disps_new = [np.array([0])]
    total_disp = 0
    for idx in path:
        disp = disps[idx]
        idx_min = np.argmin(disp[:max_config])
        total_disp += disp[idx_min]
        qs_new += [qs[idx][idx_min]]
        disps_new += [disps[idx][idx_min]]
    disps_new += [np.array([0])]
    qs_new += [q_init[0]]
    toc = time.time()

    comp_time.select_config = toc-tic
    comp_time.plan = 0
    comp_time.postplan = 0

    
    clear_output()
    print('Plan the motion')
    trajs = []
    total_dist = 0
    for i in range(len(path)-1):
        q = qs_new[i]
        qn = qs_new[i+1]
        tic = time.time()
        traj,_,_ = rrt.plan(q,qn)
        toc = time.time()
        comp_time.plan += toc-tic
        
        shortpath = rrt.shortcut_path(traj, max_try = max_try)
        shorttraj = np.array(traj)[shortpath]
        shortpath2 = rrt.shortcut_path_old(shorttraj,step_length=0.05)
        shorttraj2 = rrt.interpolate_traj(shortpath2, shorttraj)
        #shorttraj2 = rrt.interpolate_traj(shortpath, traj)
        toc2 = time.time()
        comp_time.postplan += toc2-toc
        trajs += [shorttraj2]
        total_dist += calc_dist(shorttraj2)
        clear_output()


    clear_output()
    print('Planning time:{}, {}, {}'.format(comp_time.select_config, comp_time.plan, comp_time.postplan))
    return trajs, total_dist, total_disp, comp_time

#### Determine the weight

#### Plan using RTSP

In [ ]:
num_configs = [1, 5, 10, 20, 50, 75, 100, 200]
num_configs = num_configs[6:7]

In [ ]:
rtsp_fixed_dists = dict()
rtsp_fixed_disps = dict()
rtsp_fixed_trajs = dict()
rtsp_fixed_time = dict()

In [ ]:
path2 = np.arange(len(path))

w_disp = 2000

In [ ]:
comp_time = time_struct()
tic = time.time()
print('Obtaining configurations in the required order')
qs_new = [q_init]
disps_new = [np.array([0])]
for idx in path2:
    qs_new += [qs[idx][:max_config]]
    disps_new += [disps[idx][:max_config]]
qs_new += [q_init]
disps_new += [np.array([0])]

print("Construct cgraph")
cgraph = construct_cgraph(qs_new, disps_new, w_disp = w_disp)


In [ ]:
set_q_std(q[0])

In [ ]:
set_q_std(qn[0])

In [ ]:
rrt.init_plan(q[0], qn[0])

In [ ]:
# Sample random configs
q_rand = rrt.sample()[0]

In [ ]:
# find a nearest node
rrt.G = rrt.G0
rrt.samples = rrt.samples0
nearest_index_, nearest_sample = rrt.find_nearest(q_rand, np.array(rrt.samples))
nearest_index = nearest_index_[0]

In [ ]:
rrt.check_collision(q[0])

In [ ]:
set_q_std(q_rand.flatten())

In [ ]:
set_q_std(nearest_sample.flatten())

In [ ]:
rrt.extend(nearest_index, nearest_sample.flatten(), q_rand)

In [ ]:
step_length = 0.1

In [ ]:
sample1, sample2 = nearest_sample.flatten(), q_rand
state1, state2 = sample1.copy(), sample2.copy()
dist = np.linalg.norm(state2-state1)
N = int(dist/step_length) + 2
state_list = rrt.interpolate(state1, state2, N)

In [ ]:
set_q_std(state1)

In [ ]:
rrt.check_collision(state1)

In [ ]:
set_q_std(state_list[0])

In [ ]:
for n in num_configs:
    print(n)
    print('Planning with {} configs'.format(n))
    rtsp_fixed_trajs[n], rtsp_fixed_dists[n], rtsp_fixed_disps[n], rtsp_fixed_time[n] = plan_motion(q_init, np.arange(len(path)), qs, disps, n, w_disp = 2000.)

In [ ]:
set_q_std(q_init[0])

In [ ]:
for traj in rtsp_fixed_trajs[100]:
    vis_traj_std(traj,dt=0.02)
    time.sleep(0.1)

In [ ]:


rtsp_fixed_dists[100], rtsp_fixed_disps[100]/84

In [ ]:
set_q_std(rtsp_fixed_trajs[n][1][0])

In [ ]:
rtsp_fixed_config_time = []
rtsp_fixed_plan_time = []
rtsp_fixed_postplan_time = []
for n in num_configs:
    rtsp_fixed_config_time += [rtsp_fixed_time[n].select_config]
    rtsp_fixed_plan_time += [rtsp_fixed_time[n].plan]
    rtsp_fixed_postplan_time += [rtsp_fixed_time[n].postplan]

In [ ]:
data = dict()
data['rtsp_fixed'] = [rtsp_fixed_dists, rtsp_fixed_disps, rtsp_fixed_trajs, rtsp_fixed_config_time,rtsp_fixed_plan_time, rtsp_fixed_postplan_time ]
data['num_configs'] = num_configs
np.save('data/rivet_planning_cylindrical' + str(len(qs)) + '.npy', data, allow_pickle=True)

#### visualize saved data

In [ ]:
data_loaded = np.load('data/rivet_planning_cylindrical84.npy', allow_pickle=True)

In [ ]:
loaded_traj = data_loaded.item()["rtsp_fixed"][2][100]

In [ ]:
for traj in loaded_traj:
    vis_traj_std(traj,dt=0.02)
    time.sleep(0.1)